In [ ]:
! pip install datasets -q
! pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import pandas as pd

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

batch_size = 8
num_epochs = 3

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"])

history = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 6))
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from transformers import TFAutoModelForSequenceClassification

# Re-instantiate the model
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Create Adam optimizer with custom learning rate
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# Pass the uninstantiated optimizer class
model.compile(
    optimizer=custom_optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

cb_lr = LearningRateScheduler(scheduler)

history_lr = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3,
    callbacks=[cb_lr]
)

## Training with Trainer Class

In [ ]:
! pip install datasets -q
! pip install evaluate -q

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# define a TranArgument to supply the hyperparamters
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

# now we need the model to train
# find a base model
# supply the number of classes
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# now we can train the model using Trainer
# Trainer expect
# 1. model, training_arg, trainnig and validation data, data_collator, and tokenizer

from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.504300
1000,0.269700


TrainOutput(global_step=1377, training_loss=0.31747924095491786, metrics={'train_runtime': 214.4412, 'train_samples_per_second': 51.315, 'train_steps_per_second': 6.421, 'total_flos': 405114969714960.0, 'train_loss': 0.31747924095491786, 'epoch': 3.0})

- we didn't specified the evaluation strategy
  - 'steps' evaluate on every step
  - 'epoch' on every epoch
- compute metrics
  - compute_metrics() function to calculate the metrics during evaluation

In [ ]:
import numpy as np
predictions = trainer.predict(tokenized_datasets['validation'])
preds = np.argmax(predictions.predictions, axis=-1)

import evaluate
metric = evaluate.load('glue', 'mrpc')
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8382352941176471, 'f1': 0.8869863013698631}

In [ ]:
def compute_metrics(eval_preds):
  metrics = evaluate.load('glue', 'mrpc')
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis = -1)
  return metrics.compute(predictions=predictions, references=labels)

# now need to define the evaluation_strategy in the trinaing_arg
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# need to specify the compute_metrics
# need to pass the function we have created
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.514786,0.776961,0.858034
2,0.504300,0.514033,0.845588,0.892675
3,0.269700,0.780903,0.838235,0.886986


TrainOutput(global_step=1377, training_loss=0.31747924095491786, metrics={'train_runtime': 229.5034, 'train_samples_per_second': 47.947, 'train_steps_per_second': 6.0, 'total_flos': 405114969714960.0, 'train_loss': 0.31747924095491786, 'epoch': 3.0})

## Full full training
### Process the data

In [ ]:
! pip install datasets -q
! pip install evaluate -q

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

- now need to remove extra columns
- rename columns as per model expectation
- convert the datasets to pytorch tensors

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

- data loader for supplying data to the model

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator
)

# to check the data
for batch in train_dataloader:
  break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 72]),
 'token_type_ids': torch.Size([8, 72]),
 'attention_mask': torch.Size([8, 72])}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# just to check the model output
output = model(**batch)
print(output.loss, output.logits.shape)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(0.8999, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


- Now need to define
  - optimizer
  - loss does not required, the model output has loss in it

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

# need a learning rate scheduler
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# training loop
import torch
from tqdm.auto import tqdm
import evaluate

# model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# evaluation metics
metric = evaluate.load('glue', 'mrpc')

# prograss bar
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    # now we need to compute the metrics for each batch
    # and consolidate the metrics for each batch
    metric.add_batch(predictions=predictions, references=batch['labels'])
  metric.compute()# calculate the metrics for the full dataset

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

## Accelerate library
- multiple CPU or GPU or TPU (distributed) trainnig
- Accelerator object
  - prepares the environment for training in a distributed way
- now need to prepare the objects for distributed training (accelerator.prepare())
  - model
  - data_loaders
  - optimizer
- loss.backward() -> accelerator.backward(loss)

In [ ]:
from accelerate import Accelerator # acclerator lib
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator() # addtion

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

# no need to specify the device
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

# need to change data_loders, model and optimizer for multi device training
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
      # no need to send the data to device
#       batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
        # no need to compute the loss derivatives
#       loss.backward()
# need to compute the loss based on multiple device
      accelerator.backward(loss)

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar.update(1)